<a href="https://colab.research.google.com/github/louissiller/mgmt467-analytics-portfolio/blob/main/Lab_2_VertexAI_BigQuery_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Vertex AI–Assisted BigQuery Analytics
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Copy Schema to a dataframe

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'mgmt467-71800' # This is derived from your provided table name
dataset_id = 'lab1_foundation'
table_id = 'superstore'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [ ]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like


create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `mgmt467-71800.lab1_foundation.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  Country AS country,
  City AS city,
  State AS state,
  `Postal Code` AS postal_code,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit
FROM
  `mgmt467-71800.lab1_foundation.superstore`;



## Generate View with standard column naming convention

In [ ]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



View 'superstore_clean' created/replaced successfully in dataset 'lab1_foundation'.

--- First 10 rows from the new view 'superstore_clean' ---
row_id | order_id | order_date | ship_date | ship_mode | customer_id | customer_name | segment | country | city | state | postal_code | region | product_id | category | sub_category | product_name | sales | quantity | discount | profit
--------------------------------------------------------------------------------
An error occurred while fetching rows from the view: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/mgmt467-71800/datasets/lab1_foundation/tables/superstore_clean/data?maxResults=10&formatOptions.useInt64Timestamp=True&prettyPrint=false: Cannot list a table of type VIEW.


In [ ]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

# Update table_id to point to the newly created view
table_id = new_view_id
print(f"Switched to using view: `{table_id}` for subsequent queries.")

print("\n✅ Step 1: Defining the query string...")

# The query now uses the table_id variable to reference the clean view
query_string = f"""
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `{project_id}.{dataset_id}.{table_id}`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print(f"\n⚠️ The query ran successfully but returned an empty result. Please double-check that your '{table_id}' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

Switched to using view: `superstore_clean` for subsequent queries.

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-47888.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

Results matched my expectations, except the number of the rows as I do not think that they are required.

In [ ]:
print("✅ Step 1: Defining the query string...")
query_string = f"""
SELECT
    DISTINCT sub_category
FROM
     `{project_id}.{dataset_id}.{table_id}`
WHERE
    region = 'West'
ORDER BY
    sub_category ASC
LIMIT 100
"""
print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 17 rows.

--- Displaying Results ---


,sub_category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [ ]:
query_string = f"""
SELECT
    customer_id,
    SUM(profit) AS total_profit
  FROM
     `{project_id}.{dataset_id}.{table_id}`
  GROUP BY
    customer_id
ORDER BY
  total_profit DESC
LIMIT 10
"""
# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)
except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,customer_id,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055
5,TA-21385,4703.7883
6,CM-12385,3899.8904
7,KD-16495,3038.6254
8,AR-10540,2884.6208
9,DR-12940,2869.0760


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

In [ ]:
query_string = f"""
SELECT
    ship_mode,
    COUNT(*) AS order_count
  FROM
     `{project_id}.{dataset_id}.{table_id}`
  WHERE
    category = 'Technology'
  GROUP BY
    ship_mode
ORDER BY
  order_count DESC
  """
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,ship_mode,order_count
0,Standard Class,1082
1,Second Class,366
2,First Class,301
3,Same Day,98


Here are two quick sanity checks to verify the numbers:

1.  **Total count verification:** The sum of `order_count` for all ship modes in the output should equal the total number of orders in the 'Technology' category. You can verify this by running:
    ```sql
    SELECT COUNT(*) FROM `mgmt467-71800.lab1_foundation.superstore_clean` WHERE category = 'Technology'
    ```

2.  **Spot-check a single ship mode:** Pick one `ship_mode` from the results (e.g., 'Standard Class') and run a targeted query to ensure its count matches.
    ```sql
    SELECT COUNT(*) FROM `mgmt467-71800.lab1_foundation.superstore_clean` WHERE category = 'Technology' AND ship_mode = 'Standard Class'
    ```

## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [ ]:
query_string = f"""
SELECT
  FORMAT_DATE('%Y-%m', DATE_TRUNC(order_date, MONTH)) AS year_month,
  SUM(sales) AS monthly_revenue
FROM
  `{project_id}.{dataset_id}.{table_id}`
WHERE
  order_date >= (
    SELECT
      DATE_SUB(MAX(order_date), INTERVAL 12 MONTH)
    FROM
      `{project_id}.{dataset_id}.{table_id}`
  )
GROUP BY
  year_month
ORDER BY
  year_month ASC
LIMIT 12
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print(f"Query finished. Found {len(results_df)} rows.")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

Query finished. Found 12 rows.


,year_month,monthly_revenue
0,2016-12,1505.5420
1,2017-01,43971.3740
2,2017-02,20301.1334
3,2017-03,58872.3528
4,2017-04,36521.5361
5,2017-05,44261.1102
6,2017-06,52981.7257
7,2017-07,45264.4160
8,2017-08,63120.8880
9,2017-09,87866.6520


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

In [ ]:
query_string = f"""
SELECT
    sub_category,
    SUM(profit) AS total_profit
FROM
    `{project_id}.{dataset_id}.{table_id}`
GROUP BY
    sub_category
HAVING
    total_profit < 0
ORDER BY
    total_profit ASC
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print(f"Query finished. Found {len(results_df)} rows.")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

Query finished. Found 3 rows.


,sub_category,total_profit
0,Tables,-17725.4811
1,Bookcases,-3472.5560
2,Supplies,-1189.0995


`HAVING` is used here because it filters the results *after* the `GROUP BY` and aggregation (`SUM(Profit)`) have been applied, whereas a `WHERE` clause is applied *before* aggregation.

## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [ ]:
query_string = f"""
WITH products_dimension AS (
  SELECT DISTINCT
    product_id,
    product_name
  FROM
    `{project_id}.{dataset_id}.{table_id}`
)
SELECT
  s.product_id,
  p.product_name,
  SUM(s.sales) AS total_sales
FROM
  `{project_id}.{dataset_id}.{table_id}` AS s
JOIN
  products_dimension AS p
ON
  s.product_id = p.product_id
GROUP BY
  s.product_id,
  p.product_name
ORDER BY
  total_sales DESC
LIMIT 20
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print(f"Query finished. Found {len(results_df)} rows.")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

Query finished. Found 20 rows.


,product_id,product_name,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,61599.8240
1,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.3840
2,TEC-MA-10002412,Cisco TelePresence System EX90 Videoconferenci...,22638.4800
3,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,21870.5760
4,OFF-BI-10001359,GBC DocuBind TL300 Electric Binding System,19823.4790
5,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,19024.5000
6,TEC-CO-10001449,Hewlett Packard LaserJet 3310 Copier,18839.6860
7,TEC-MA-10001127,HP Designjet T520 Inkjet Large Format Printer ...,18374.8950
8,OFF-BI-10004995,GBC DocuBind P400 Electric Binding System,17965.0680
9,OFF-SU-10000151,High Speed Automatic Electric Letter Opener,17030.3120


Since we don't have a separate `products` dimension table, we can simulate one using a Common Table Expression (CTE). The query below first creates a temporary `products_dimension` table containing unique product IDs and names, and then joins it back to the main sales table to calculate the total sales for each product.

## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

This query operates in three main stages. First, the `state_sales` Common Table Expression (CTE) calculates the total sales for each state by grouping the data by both `Region` and `State`. Second, the `ranked_state_sales` CTE takes this aggregated data and assigns a rank to each state within its region based on its total sales, with the highest sales getting a rank of 1. Finally, the main `SELECT` statement filters these results to only include the states that ranked in the top 3 for their respective regions, and then sorts the output for clear presentation.

In [ ]:
query_string = f"""
WITH state_sales AS (
  SELECT
    Region,
    State,
    SUM(sales) AS total_sales
  FROM
    `{project_id}.{dataset_id}.{table_id}`
  GROUP BY
    Region,
    State
),
ranked_state_sales AS (
  SELECT
    Region,
    State,
    total_sales,
    RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS sales_rank
  FROM
    state_sales
)
SELECT
  Region,
  State,
  total_sales,
  sales_rank
FROM
  ranked_state_sales
WHERE
  sales_rank <= 3
ORDER BY
  Region,
  sales_rank
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print(f"Query finished. Found {len(results_df)} rows.")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

Query finished. Found 12 rows.


,Region,State,total_sales,sales_rank
0,Central,Texas,170188.0458,1
1,Central,Illinois,80166.1010,2
2,Central,Michigan,76269.6140,3
3,East,New York,310876.2710,1
4,East,Pennsylvania,116511.9140,2
5,East,Ohio,78258.1360,3
6,South,Florida,89473.7080,1
7,South,Virginia,70636.7200,2
8,South,North Carolina,55603.1640,3
9,West,California,457687.6315,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

In [ ]:
query_string = f"""
WITH yr_sales AS (
  SELECT
    sub_category,
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(sales) AS yearly_sales
  FROM
    `{project_id}.{dataset_id}.{table_id}`
  WHERE
    EXTRACT(YEAR FROM order_date) IN (2023, 2024)
  GROUP BY
    sub_category, year
)
SELECT
  sub_category,
  SUM(IF(year = 2023, yearly_sales, 0)) AS sales_2023,
  SUM(IF(year = 2024, yearly_sales, 0)) AS sales_2024,
  SUM(IF(year = 2024, yearly_sales, 0)) - SUM(IF(year = 2023, yearly_sales, 0)) AS yoy_delta
FROM
  yr_sales
GROUP BY
  sub_category
ORDER BY
  yoy_delta DESC
LIMIT 5;
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print(f"Query finished. Found {len(results_df)} rows.")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

Query finished. Found 0 rows.


,sub_category,sales_2023,sales_2024,yoy_delta


### Failure Modes and Solutions

1.  **Failure Mode: Sub-category exists in one year but not the other.** If a sub-category has sales in 2024 but not in 2023 (e.g., a new product line), an `INNER JOIN` approach would exclude it from the results.
    *   **Solution:** The query above handles this correctly by using conditional aggregation (`SUM(IF(...))`), which treats the missing year's sales as zero, ensuring new or discontinued sub-categories are properly included in the YoY calculation.

2.  **Failure Mode: Entire dataset is missing data for one or both years.** If the source data does not contain any records for 2023 or 2024, the query would run without error but produce misleading results (e.g., all `yoy_delta` values would be zero or negative).
    *   **Solution:** Before running the analysis, perform a quick validation query to ensure data exists for the target years, like `SELECT DISTINCT EXTRACT(YEAR FROM order_date) FROM your_table`. This confirms the presence of the necessary data before you proceed with the more complex YoY calculation.

## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

In [ ]:
query_string = f"""
WITH subcat_sales AS (
  SELECT
    region,
    sub_category,
    SUM(sales) AS total_sales
  FROM
    `{project_id}.{dataset_id}.{table_id}`
  GROUP BY
    region,
    sub_category
),
ranked_sales AS (
  SELECT
    region,
    sub_category,
    total_sales,
    ROW_NUMBER() OVER (PARTITION BY region ORDER BY total_sales DESC) as rn
  FROM
    subcat_sales
)
SELECT
  region,
  sub_category,
  total_sales
FROM
  ranked_sales
WHERE
  rn = 1
ORDER BY
  region
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print(f"Query finished. Found {len(results_df)} rows.")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

Query finished. Found 4 rows.


,region,sub_category,total_sales
0,Central,Chairs,85230.646
1,East,Phones,100614.982
2,South,Phones,58304.438
3,West,Chairs,101781.328


`ROW_NUMBER()` is used here because it guarantees exactly one row per region by assigning a unique, sequential number to each row, arbitrarily breaking any ties. In contrast, `RANK()` would assign the same rank to sub-categories with identical sales, potentially causing the query to return more than one 'top' sub-category per region if a tie exists.

### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [ ]:
query_string = f"""
WITH phones_yearly_sales AS (
  SELECT
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(sales) AS yearly_revenue
  FROM
    `{project_id}.{dataset_id}.{table_id}`
  WHERE
    sub_category = 'Phones'
  GROUP BY
    year
)
SELECT
  year,
  yearly_revenue,
  LAG(yearly_revenue) OVER (ORDER BY year) AS prev_revenue,
  -- Guard against divide-by-zero for the first year
  SAFE_DIVIDE(yearly_revenue - LAG(yearly_revenue) OVER (ORDER BY year), LAG(yearly_revenue) OVER (ORDER BY year)) * 100 AS yoy_pct
FROM
  phones_yearly_sales
ORDER BY
  year ASC
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print(f"Query finished. Found {len(results_df)} rows.")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

Query finished. Found 4 rows.


,year,yearly_revenue,prev_revenue,yoy_pct
0,2014,77390.806,NaN,NaN
1,2015,68313.702,77390.806,-11.728918
2,2016,78962.030,68313.702,15.587397
3,2017,105340.516,78962.030,33.406545


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [ ]:
# Cost-control note: For faster iteration during development, add a WHERE clause to filter for a smaller date range.
query_string = f"""
WITH monthly_sales AS (
  SELECT
    DATE_TRUNC(order_date, MONTH) AS month,
    SUM(sales) AS monthly_revenue
  FROM
    `{project_id}.{dataset_id}.{table_id}`
  WHERE
    segment = 'Corporate'
  GROUP BY
    month
)
SELECT
  month,
  monthly_revenue,
  AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS ma_3
FROM
  monthly_sales
ORDER BY
  month ASC
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print(f"Query finished. Found {len(results_df)} rows.")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

Query finished. Found 48 rows.


,month,monthly_revenue,ma_3
0,2014-01-01,1701.5280,1701.528000
1,2014-02-01,1183.6680,1442.598000
2,2014-03-01,11106.7990,4663.998333
3,2014-04-01,14131.7290,8807.398667
4,2014-05-01,9142.0000,11460.176000
5,2014-06-01,3970.9140,9081.547667
6,2014-07-01,10032.9880,7715.300667
7,2014-08-01,7451.7740,7151.892000
8,2014-09-01,15507.7450,10997.502333
9,2014-10-01,12637.6780,11865.732333


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
[PASTE ERROR MESSAGE and the exact SQL here]
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```
***Remark:*** My prompt led to a working code, therefore I am skipping this task and keep it for a future project where errors occur.

### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[PASTE YOUR SQL HERE]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

Here are three primary ways to reduce scanned bytes and improve BigQuery performance:

1.  **Prune Columns with `SELECT`:** Instead of using `SELECT *`, explicitly list only the columns you need. Since BigQuery is a columnar database, this is the most direct way to reduce the amount of data read from disk. For example, use `SELECT order_id, sales` instead of `SELECT *` if you only need those two fields.

2.  **Apply Filters on Partition and Cluster Keys Early:** If your table is partitioned (e.g., by date), always include a `WHERE` clause that filters on the partition column (e.g., `WHERE order_date BETWEEN '2023-01-01' AND '2023-03-31'`). This tells BigQuery to only scan the relevant partitions, drastically reducing data processed. If the table is also clustered (e.g., by `Region`), filtering on that key further improves performance by allowing BigQuery to avoid reading entire blocks within the selected partitions.

3.  **Create and Query Pre-Aggregated Summary Tables:** For dashboards or frequent analytical queries, it's highly efficient to pre-aggregate raw data into smaller, summary tables. For example, instead of calculating total daily sales from a massive transaction table every time, you can run a single daily job to create a `daily_sales_summary` table. Subsequent queries against this much smaller summary table will be significantly faster and cheaper.

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

```sql
-- Alternative 1: Slice by Region and Time
-- This query checks if the negative profit for 'Tables' is consistent across all regions and years.
SELECT
  region,
  EXTRACT(YEAR FROM order_date) AS order_year,
  SUM(sales) AS total_sales,
  SUM(profit) AS total_profit,
  AVG(discount) AS average_discount
FROM
  `mgmt467-71800.lab1_foundation.superstore_clean`
WHERE
  sub_category = 'Tables'
GROUP BY
  region,
  order_year
ORDER BY
  region,
  order_year;
```

```sql
-- Alternative 2: Control for Ship Mode
-- This query investigates if specific shipping methods are disproportionately contributing to the negative profit.
SELECT
  ship_mode,
  COUNT(*) AS order_count,
  SUM(sales) AS total_sales,
  SUM(profit) AS total_profit,
  AVG(discount) AS average_discount
FROM
  `mgmt467-71800.lab1_foundation.superstore_clean`
WHERE
  sub_category = 'Tables'
GROUP BY
  ship_mode
ORDER BY
  total_profit ASC;
```

To analyze these results, first run both queries. Compare the outcomes from the region/time query: if 'Tables' are profitable in certain regions or were profitable in past years, it suggests the problem is not inherent to the product but may be linked to regional pricing strategies, local competition, or recent changes in costs. Next, examine the ship mode query: if a specific shipping class (e.g., 'Same Day') shows a significantly larger negative profit margin compared to others, it could indicate that the freight costs for that service are a primary driver of the losses for this bulky sub-category, challenging the initial conclusion that discounts alone are to blame.

## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

In [ ]:
query_string = f"""
WITH yearly_summary AS (
  -- Step 1: Aggregate sales and profit for the last two full years by sub-category
  SELECT
    sub_category,
    EXTRACT(YEAR FROM order_date) AS sales_year,
    SUM(sales) AS total_sales,
    SUM(profit) AS total_profit
  FROM
    `{project_id}.{dataset_id}.{table_id}`
  WHERE
    EXTRACT(YEAR FROM order_date) IN (2016, 2017) -- Focus on the last two full years
  GROUP BY
    sub_category,
    sales_year
),
-- Step 2: Pivot the data to compare years side-by-side
pivot_by_year AS (
  SELECT
    sub_category,
    SUM(IF(sales_year = 2016, total_sales, 0)) AS sales_2016,
    SUM(IF(sales_year = 2017, total_sales, 0)) AS sales_2017,
    SUM(IF(sales_year = 2016, total_profit, 0)) AS profit_2016,
    SUM(IF(sales_year = 2017, total_profit, 0)) AS profit_2017
  FROM
    yearly_summary
  GROUP BY
    sub_category
)
-- Step 3: Calculate the year-over-year change to find the biggest drivers
SELECT
  sub_category,
  sales_2016,
  sales_2017,
  profit_2016,
  profit_2017,
  (sales_2017 - sales_2016) AS sales_change,
  (profit_2017 - profit_2016) AS profit_change
FROM
  pivot_by_year
ORDER BY
  profit_change DESC -- Sort by the biggest profit impact (positive or negative)
"""

try:
    print("Running query to identify key business drivers for the executive summary...")
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print(f"Query finished. Found {len(results_df)} rows.")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

Running query to identify key business drivers for the executive summary...
Query finished. Found 17 rows.


,sub_category,sales_2016,sales_2017,profit_2016,profit_2017,sales_change,profit_change
0,Copiers,49599.4100,62899.3880,17742.7926,25031.7902,13299.9780,7288.9976
1,Accessories,41895.8540,59946.2320,9664.2885,15672.3570,18050.3780,6008.0685
2,Phones,78962.0300,105340.5160,9459.5929,12849.3250,26378.4860,3389.7321
3,Paper,20661.8940,27694.7180,9071.5345,12040.8434,7032.8240,2969.3089
4,Appliances,26050.3150,42926.9320,5301.3415,7865.2683,16876.6170,2563.9268
5,Chairs,83918.6450,95554.3530,5763.1539,7643.5493,11635.7080,1880.3954
6,Storage,58788.7000,69677.6180,6204.2868,7402.8007,10888.9180,1198.5139
7,Art,5960.9080,8863.0680,1413.9626,2221.9631,2902.1600,808.0005
8,Labels,2827.2400,3861.2160,1192.6119,1744.6093,1033.9760,551.9974
9,Furnishings,27874.1240,28915.0940,3935.2700,4099.1628,1040.9700,163.8928


### Executive Summary

**Act as a business strategist.**
Based on the metrics from the query above, write a 4-sentence executive summary:

*   **1. What changed and by how much:** Overall profit grew/declined by [Total Profit Change from 2016 to 2017], reaching [Total Profit in 2017].
*   **2. Why it likely changed (drivers):** This change was primarily driven by significant profit growth in the [Sub-Category with highest positive `profit_change`] sub-category, which helped offset losses from [Sub-Category with highest negative `profit_change`].
*   **3. Recommended action (who/what/when):** We recommend the marketing team create a targeted campaign for [Sub-Category with highest positive `profit_change`] starting next quarter, while the product team investigates the profitability issues in [Sub-Category with highest negative `profit_change`].
*   **4. Metric to monitor next:** The key metric to monitor will be the `profit_margin` for the [Sub-Category with highest negative `profit_change`] sub-category to track the effectiveness of our intervention.

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

In [ ]:
import os
import logging
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPICallError

# --- Basic Configuration ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def run_yoy_kpi_job(project_id: str, source_dataset: str, source_table: str, dest_dataset: str, dest_table: str, current_year: int, previous_year: int):
    """
    Runs a year-over-year KPI analysis and saves the results to a BigQuery table.

    Args:
        project_id: Your Google Cloud project ID.
        source_dataset: The dataset of the source data (e.g., 'lab1_foundation').
        source_table: The table/view of the source data (e.g., 'superstore_clean').
        dest_dataset: The destination dataset for the output (e.g., 'analytics').
        dest_table: The destination table for the output (e.g., 'outputs_kpi').
        current_year: The current year for the YoY comparison.
        previous_year: The previous year for the YoY comparison.
    """
    logging.info(f"Starting YoY KPI job for {previous_year} vs {current_year}.")
    client = bigquery.Client(project=project_id)

    destination_table_ref = f"{project_id}.{dest_dataset}.{dest_table}"

    # --- Parameterized SQL Query ---
    # This query calculates the change in sales and profit between two specified years.
    sql = f"""
    WITH yearly_summary AS (
      SELECT
        sub_category,
        EXTRACT(YEAR FROM order_date) AS sales_year,
        SUM(sales) AS total_sales,
        SUM(profit) AS total_profit
      FROM
        `{project_id}.{source_dataset}.{source_table}`
      WHERE
        EXTRACT(YEAR FROM order_date) IN ({current_year}, {previous_year})
      GROUP BY
        sub_category,
        sales_year
    ),
    pivot_by_year AS (
      SELECT
        sub_category,
        SUM(IF(sales_year = {previous_year}, total_sales, 0)) AS sales_{previous_year},
        SUM(IF(sales_year = {current_year}, total_sales, 0)) AS sales_{current_year},
        SUM(IF(sales_year = {previous_year}, total_profit, 0)) AS profit_{previous_year},
        SUM(IF(sales_year = {current_year}, total_profit, 0)) AS profit_{current_year}
      FROM
        yearly_summary
      GROUP BY
        sub_category
    )
    SELECT
      sub_category,
      (sales_{current_year} - sales_{previous_year}) AS sales_change,
      (profit_{current_year} - profit_{previous_year}) AS profit_change
    FROM
      pivot_by_year
    """

    # --- Job Configuration ---
    job_config = bigquery.QueryJobConfig(
        destination=destination_table_ref,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE, # Overwrite the table each time
    )

    try:
        logging.info(f"Executing query and writing results to {destination_table_ref}...")
        query_job = client.query(sql, job_config=job_config)
        query_job.result()  # Wait for the job to complete
        logging.info(f"Successfully created or updated {destination_table_ref}.")

        # Optional: Log the number of rows written
        destination_table = client.get_table(destination_table_ref)
        logging.info(f"Wrote {destination_table.num_rows} rows.")

    except GoogleAPICallError as e:
        logging.error(f"An API error occurred: {e}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

# --- How to Run the Script ---
if __name__ == "__main__":
    # IMPORTANT: Replace with your project and dataset details.
    # You can also get this from environment variables for better security.
    GCP_PROJECT_ID = "mgmt467-71800" # Or os.environ.get("GCP_PROJECT_ID")
    SOURCE_DATASET = "lab1_foundation"
    SOURCE_TABLE = "superstore_clean"

    # The destination for your analytics outputs
    DESTINATION_DATASET = "analytics" # Make sure this dataset exists in your project
    DESTINATION_TABLE = "outputs_kpi"

    # The years you want to compare
    ANALYSIS_CURRENT_YEAR = 2017
    ANALYSIS_PREVIOUS_YEAR = 2016

    # Before running, ensure the destination dataset 'analytics' exists in your project.
    # You can create it in the BigQuery console if it doesn't.

    run_yoy_kpi_job(
        project_id=GCP_PROJECT_ID,
        source_dataset=SOURCE_DATASET,
        source_table=SOURCE_TABLE,
        dest_dataset=DESTINATION_DATASET,
        dest_table=DESTINATION_TABLE,
        current_year=ANALYSIS_CURRENT_YEAR,
        previous_year=ANALYSIS_PREVIOUS_YEAR,
    )


ERROR:root:An API error occurred: 404 Not found: Dataset mgmt467-71800:analytics was not found in location US; reason: notFound, message: Not found: Dataset mgmt467-71800:analytics was not found in location US

Location: US
Job ID: 67948756-6a7f-485d-ba30-73caf29aa3d9



---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---